In [1]:
import dataclasses
import time

import torch
import torch.nn as nn
import wandb
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.io import decode_image
from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from torchvision.transforms import v2
from tqdm.notebook import tqdm
#from screenshot_dataset import ScreenshotDataset

In [2]:
@dataclasses.dataclass
class RunConfig:
    batch_size: int
    learning_rate: float
    momentum: float
    num_epochs: int
    architecture: str
    image_height: int
    image_width: int
config = RunConfig(
    batch_size = 8,
    learning_rate = 0.001,
    momentum = 0.0,
    num_epochs = 1000,  # This will overfit.
    architecture = "EfficientNetV2_s",
    image_height = 224,
    image_width = 224
)

In [3]:
augmentations = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size=(config.image_width, config.image_height), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.2, 0.2, 0.2]),
])

ds = ImageFolder("./bad_crop_data/", transform=augmentations)
train, validate = torch.utils.data.random_split(ds, [0.8, 0.2])
train_dataloader = DataLoader(dataset=train, batch_size=config.batch_size, num_workers=4, pin_memory=True, shuffle=True)
validation_dataloader = DataLoader(dataset=validate, batch_size=config.batch_size, num_workers=4, pin_memory=False, shuffle=False)
print(ds.classes)

['bad_crop', 'good_crop']


In [4]:
#torch.backends.quantized.engine = ""
device = torch.device("cuda")

In [5]:
class ScreenshotModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
        self.head = nn.Linear(1000, 2)
    def forward(self, x):
        return self.head(self.stem(x))

In [6]:
model = ScreenshotModel().to(device)
preprocessing = EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()
optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate, momentum=config.momentum)
loss_fn = torch.nn.CrossEntropyLoss()

dataclasses.asdict(config)
run = wandb.init(
    # Set the wandb project where this run will be logged.
    project="bad_crop",
    # Track hyperparameters and run metadata.
    config=dataclasses.asdict(config),
)

wandb: Currently logged in as: josephc to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
def train_epoch(model, data, optimizer, loss_fn):
    model.train()
    sample_count = 0
    total_loss = 0.0
    for i, (image, target) in enumerate(tqdm(data)):
        start_time = time.time()
        with torch.amp.autocast('cuda'):
            image = preprocessing(image)
            image, target = image.to(device), target.to(device)
            output = model(image)
            loss = loss_fn(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            sample_count += image.shape[0]
            total_loss += loss.item()
        end_time = time.time()
    print(f"Train epoch loss: {total_loss}\tAvg loss: {total_loss/sample_count}")
    return total_loss

def run_eval(model, data, loss_fn):
    model.eval()
    sample_count = 0
    total_loss = 0.0
    for i, (image, target) in enumerate(tqdm(data)):
        with torch.amp.autocast('cuda'):
            image = preprocessing(image)
            image, target = image.to(device), target.to(device)
            output = model(image)
            loss = loss_fn(output, target)
            total_loss += loss.item()
            sample_count += image.shape[0]
    print(f"Eval: total loss: {total_loss}\tavg loss: {total_loss/sample_count}")
    return total_loss

In [8]:
best_loss = 1e32
for epoch_idx in range(config.num_epochs):
    print(f"Epoch {epoch_idx}")
    batch_loss = train_epoch(model, train_dataloader, optimizer, loss_fn)
    validation_loss = run_eval(model, validation_dataloader, loss_fn)
    if validation_loss < best_loss:
        best_loss = validation_loss
        torch.save(model, f"screenshot_checkpoint_{epoch_idx%10}.pt")
    wandb.log({"batch_loss": batch_loss, "validation_loss": validation_loss})
run.finish()

Epoch 0


  0%|          | 0/39 [00:00<?, ?it/s]

Train epoch loss: 23.50956180691719	Avg loss: 0.07559344632449258


  0%|          | 0/10 [00:00<?, ?it/s]

Eval: total loss: 6.206304907798767	avg loss: 0.08060136243894503
Epoch 1


...

  0%|          | 0/39 [00:00<?, ?it/s]

Train epoch loss: 0.31218807166442275	Avg loss: 0.0010038201661235458


  0%|          | 0/10 [00:00<?, ?it/s]

Eval: total loss: 3.9946692045778036	avg loss: 0.051878820838672776


batch_loss,█▄▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,▄▄▅▆▄▃▄▆▃▅▆▄▆▄▁▃▅▂▅▆▇▆▄▇▅▅▆▇▄▄▃█▆▆▄▅▅▄▇▆
batch_loss,0.31219
validation_loss,3.99467


In [9]:
torch.save(model, "bad_crop.pt")

# Export and Validate:

In [10]:
# 
model.eval()
input_tensor = torch.rand((1, 3, 224, 224), dtype=torch.float32).to(device)
torch.onnx.export(
    model,                  # model to export
    (input_tensor,),        # inputs of the model,
    "bad_crop.onnx",        # filename of the ONNX model
    #dynamic_shapes=[{0: "batch"}],
    #dynamic_axes={"input":{0: "batch"}},  # When dynamo is False
    input_names=["input"],  # Rename inputs for the ONNX model
    output_names=["output"],
    dynamo=True             # True or False to select the exporter to use
)

[torch.onnx] Obtain model graph for `ScreenshotModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ScreenshotModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 220 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.9.1+cu128',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[1,3,224,224]>
            ),
            outputs=(
                %"output"<FLOAT,[1,2]>
            ),
            initializers=(
                %"stem.features.0.0.weight"<FLOAT,[24,3,3,3]>{Tensor(...)},
                %"stem.features.1.0.block.0.0.weight"<FLOAT,[24,24,3,3]>{Tensor(...)},
                %"stem.features.1.1.block.0.0.weight"<FLOAT,[24,24,3,3]>{Tensor(...)},
                %"stem.features.2.0.block.0.0.weight"<FLOAT,[96,24,3,3]>{Tensor(...)},
                %"stem.features.2.0.block.1.0.weight"<FLOAT,[48,96,1,1]>{Tensor(...)},
                %"stem.features.2.1.block.0.0.weight"<FLOAT,[192,48,3,3]>{Tensor(...)},
              

In [11]:
# Convert the two-part onnx + onnx data pair into a self-contained ONNX.
# Our model should be below the 2GB boundary for protos.
import onnx
onnx_model = onnx.load("bad_crop.onnx")
onnx.save_model(onnx_model, "bad_crop.onnx", save_as_external_data=False, all_tensors_to_one_file=True)

In [12]:
import numpy
import onnxruntime as ort
session = ort.InferenceSession("./screenshot.onnx")
blank = numpy.zeros((1, 3, 224, 224), dtype=numpy.float32)
out = torch.softmax(torch.tensor(session.run(["output"], {"input": blank,})[0][0]), axis=0)
print(out.shape)
print(out)

torch.Size([2])
tensor([0.0377, 0.9623])
